In [2]:
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import re
from snowballstemmer import TurkishStemmer
import string
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import news
import fasttext
import fasttext.util
fasttext.util.download_model('tr', if_exists='ignore')
ft = fasttext.load_model('cc.tr.300.bin')

In [3]:
def stem_it(x):
    """Turkish stemmer.
    Returns stemmed words in given string, uses Snowball Turkish stemmer."""
    return [turkStem.stemWord(w) for w in x.split()]

def remove_digit(text):
    """Digit cleaner.
    Removes digits in given string."""
    digit_pattern = str.maketrans('', '', string.digits)
    return text.translate(digit_pattern)

def lower_it(text):
    """Lowers characters in given string.
    Lowers all in string."""
    return text.lower()

def remove_punctiations(text):
    """Punction cleaner.
    Removes punctions in given string."""
    pattern = str.maketrans('', '', string.punctuation)
    text= text.translate(pattern)
    return text.replace("\n", " ")

def remove_stopwords(text, language='turkish'):
    """Stopword cleaner.
    Removes stopwords in given string, language is set to Turkish by default."""
    st = stopwords.words(language)
    pattern = re.compile(r'\b(' + r'|'.join(st) + r')\b\s*')
    text = pattern.sub('',text)
    return text

def replace_turkish(text):
    """Turkish Character Fixer.
    Replaces Turkish character with English ones."""
    character_map = {'ç': 'c',
                     'Ç': 'C',
                     'ğ': 'g',
                     'Ğ': 'G',
                     'ı': 'i',
                     'I': 'I',
                     'ö': 'o',
                     'Ö': 'O',
                     'ş': 's',
                     'Ş': 'S', 
                     'ü': 'u',
                     'Ü': 'U'}
    for initial, final in character_map.items():
        text = text.replace(initial, final)
    return text

def sentence_it(x):
    """Sentence maker.
    Returns one string form of given words array."""
    sentence = ''
    for w in x.split(): sentence += w + " "
    return sentence.strip()


In [4]:

def preproces(text, stemmer=False):
    """String cleaner.
    Removes digits, lowers charactes, removes punctiations and stop words in given string."""
    text = remove_digit(text)
    text = lower_it(text)
    text = remove_punctiations(text)
    text = remove_stopwords(text)
    #text = replace_turkish(text) # Since FastText includes Turkish characters, there is no need to replace them.
    # But for different libraries, one may has to change them.
    if stemmer:
        text = stem_it(text)
        text = sentence_it(text)
    text = sentence_it(text)
    return text

In [5]:
def text_cleaning(text):
    """Text cleaner.
    Returns sentences and cleaned sentences."""
    sentences = sent_tokenize(text)
    clean_sentenes = [preproces(sentence) for sentence in sentences]
    return sentences, clean_sentenes 

In [6]:
def sentence_vector(sentence):
    return sum([ft.get_word_vector(word) for word in sentence])/(len(sentence)+0.0001)

In [7]:
def word_vectors(text):
    """Word vector creator.
    Uses cleaned text array and returns word vector array"""
    sentence_words = [[w for w in s.split()] for s in text]
    word_vectors = [sentence_vector(s_words) for s_words in sentence_words]
    return word_vectors

In [8]:
def similarity_matrix(text_vector):
    """Similarty Matrix generator.
    Creates similarity matrix using sentence vectors via cosine similarity"""
    sim_mat = np.zeros([len(text_vector), len(text_vector)])
    for i in range(len(text_vector)):
        for j in range(len(text_vector)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(text_vector[i].reshape(1, -1), text_vector[j].reshape(1, -1))[0, 0]
    return sim_mat

In [9]:
def extraction(sentences, similarity_matrix):
    nx_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(nx_graph)
    text_dict = {}
    for i in range(len(sentences)):
        text_dict.update({str(i):{"score": scores[i],
                                  "text": sentences[i]}})
    text_order = sorted(text_dict, key=lambda x: text_dict[x]["score"], reverse=True)
    extract = ""
    for i in range(int(len(sentences)/3)):
        id = text_order[i]
        extract = extract + " " + text_dict[id]["text"]
    return extract

In [10]:
def generate_extraction(text):
    sentences, cleaned_sentences = text_cleaning(text)
    sentence_vectors = word_vectors(cleaned_sentences)
    sim_mat = similarity_matrix(sentence_vectors)
    return extraction(sentences, sim_mat)

In [16]:
article = news.get_article()
print(article)

İsrail İran’ın‘misilleme’saldırısına karşılık vermekten‘son anda’vazgeçti. İran, Suriye’deki büyükelçilik binasını vuran İsrail’e karşı‘misilleme’saldırısı başlatmış, İsrail’e yüzlerce insansız hava aracı  ve füze atılmıştı. İran‘Şimdilik bu kadar’mesajı verince gerilim korkulanın aksinedüşmüştü. İsrail’in İran’ın misillemesinden neredeyse hiç zarar görmemesi gerilimi daha da azalttı.  Yine de İsrail’in olası bir karşı saldırısının gerilimi yeniden tırmandırmasından kaygı duyuluyordu. ABD Başkanı Joe Biden’ın saldırıdan sonra telefonla görüştüğü İsrail Başbakanı Binyamin Netanyahu’dan herhangi bir karşılık verilmemesini istediği öne sürülmüştü. İsrail devlet televizyonu KAN, Netanyahu yönetiminin İran’ın saldırılarına karşılık vermekten‘son anda’bildirdi. Savaş kabinesi ve bakanlar kurulunun önce salt çoğunlukla İran’a derhal karşılık verilmesini onayladığı ancak Biden’la görüşme sonrası bu kararın değiştiği kaydedildi. 


In [17]:
generate_extraction(article)

' Yine de İsrail’in olası bir karşı saldırısının gerilimi yeniden tırmandırmasından kaygı duyuluyordu. Savaş kabinesi ve bakanlar kurulunun önce salt çoğunlukla İran’a derhal karşılık verilmesini onayladığı ancak Biden’la görüşme sonrası bu kararın değiştiği kaydedildi.'